# Operations Reasearch: Examples for Gurobipy

This code is for students who take the class Operations Research.
Students should finish the installation of Gurobi and Python before started and make sure an academic liscense for Gurobi is applied and activated.

We introduce an example for integer programming problem in order to help students understand how to solve optimization programs with codes.
More insturction is provided in the lecture video.

## Facility location
In this example, we have binary variables included in our model. The purpose of this section is to help student understand how to solve integer programs by Gurobi.

The problem description is provided in the lecture video and slides. The formulation is
\begin{array}{rll}
                \min & \displaystyle \sum_{j=1}^{5} f_jx_j + \sum_{i=1}^{5} \sum_{j=1}^{5} c_{ij}y_{ij} & \\[15pt]
                \mbox{s.t.}  
                    & \displaystyle \sum_{i=1}^{5} y_{ij} \leq K_jx_j & \forall j = 1,...,5 \\[15pt]
                    & \displaystyle \sum_{j=1}^{5} y_{ij} \geq D_i & \forall i = 1,...,5 \\[15pt]
                    & x_{j} \in \{0, 1\} &\forall j = 1,...,5 \\[5pt]
                    & y_{ij} \geq 0 & \forall i = 1,...,5, j = 1,...,5.
\end{array}

In [1]:
from gurobipy import *
import pandas as pd

In [2]:
# Read excel sheets and transform them into lists and matrices
basic_info = pd.read_excel('IP_dataset.xlsx', 'Basic information')
cities = range(len(basic_info['City']))
markets = range(len(basic_info['Market']))

city_info = pd.read_excel('IP_dataset.xlsx', 'City\'s information')
operating_costs = city_info['Operating cost']
capacities = city_info['Capacity']

market_info = pd.read_excel('IP_dataset.xlsx', 'Market\'s information')
demands = market_info['Demand']

shipping_info = pd.read_excel('IP_dataset.xlsx', 'Shipping cost', index_col = 0)
shipping_costs = []
for i in shipping_info.index:
    shipping_costs.append(list(shipping_info.loc[i]))

In [3]:
eg2 = Model("eg2")    # build a new model
    
# add variables as a list
x = []
for j in cities:
    x.append(eg2.addVar(lb = 0, vtype = GRB.BINARY, name = "x" + str(j+1)))
             
y = []
for i in markets:
    y.append([])
    for j in cities:
        y[i].append(eg2.addVar(lb = 0, vtype = GRB.CONTINUOUS, name = "y" + str(i+1) + str(j+1)))
        
# setting the objective function 
eg2.setObjective(
    quicksum(operating_costs[j] * x[j] for j in cities) 
    +  quicksum(quicksum(shipping_costs[i][j] * y[i][j] for j in cities) for i in markets)
    , GRB.MINIMIZE) 

# add constraints and name them
eg2.addConstrs((quicksum(y[i][j] for i in markets) <= capacities[j] * x[j] 
                for j in cities), "productCapacity")

eg2.addConstrs((quicksum(y[i][j] for j in cities) >= demands[i]
                for i in markets), "demand_fulfillment")
    
eg2.optimize()

Using license file C:\Users\user\gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (win64)
Optimize a model with 10 rows, 30 columns and 55 nonzeros
Model fingerprint: 0x263201cb
Variable types: 25 continuous, 5 integer (5 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+04]
  Objective range  [2e+00, 4e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [8e+03, 2e+04]
Presolve time: 0.00s
Presolved: 10 rows, 30 columns, 55 nonzeros
Variable types: 25 continuous, 5 integer (5 binary)

Root relaxation: objective 2.528000e+05, 11 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 252800.000    0    5          - 252800.000      -     -    0s
H    0     0                    313600.00000 252800.000  19.4%     -    0s
H    0     0                    280400.00000 252800.000  9.84%   

In [4]:
print("Result:")

for j in cities:
    print(x[j].varName, '=', x[j].x)
# head of the result table
print("\tMarket1\tMarket2\tMarket3\tMarket4\tMarket5")

for j in cities:
    # mark which product is printed now
    print("City" + str(j+1), "\t", end="")
    for i in markets:
        # print values of each kind of product
        if len(str(y[i][j].x)) < 7:
            print(y[i][j].x, "\t", end="")
        else:
            print(y[i][j].x, "", end="")
    print("")    # use for change line

print("z* =", eg2.objVal)    # print objective value

Result:
x1 = 0.0
x2 = 1.0
x3 = 0.0
x4 = 1.0
x5 = 1.0
	Market1	Market2	Market3	Market4	Market5
City1 	0.0 	0.0 	0.0 	0.0 	0.0 	
City2 	8000.0 	12000.0 0.0 	0.0 	0.0 	
City3 	0.0 	0.0 	0.0 	0.0 	0.0 	
City4 	0.0 	0.0 	8000.0 	0.0 	17000.0 
City5 	0.0 	0.0 	1000.0 	14000.0 0.0 	
z* = 268950.0


***Exercise*** Suppose now the company wants to set up at least 4 centers for some reasons, how should we modify the model?